In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.signal import butter, filtfilt
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_recall_curve
from sklearn.metrics import classification_report, precision_score, recall_score
from sklearn.feature_selection import SelectFromModel
import joblib


np.random.seed(42)
#  Data Loading and Exploration
#dummy
def generate_sample_data(n_samples=10000):
  
    timestamps = pd.date_range(start='2025-01-01', periods=n_samples, freq='100ms')
    
   
    activities = ['walking', 'running', 'resting', 'playing']
    activity_labels = np.random.choice(activities, size=n_samples, 
                                     p=[0.4, 0.2, 0.3, 0.1])  
    

    acc_x, acc_y, acc_z = [], [], []
    
    for activity in activity_labels:
        if activity == 'walking':
            acc_x.append(np.sin(np.random.rand() * np.pi) + np.random.normal(0, 0.2))
            acc_y.append(np.cos(np.random.rand() * np.pi) + np.random.normal(0, 0.3))
            acc_z.append(np.sin(np.random.rand() * np.pi) + np.random.normal(0, 0.2))
        elif activity == 'running':
            acc_x.append(1.5 * np.sin(np.random.rand() * np.pi) + np.random.normal(0, 0.4))
            acc_y.append(1.5 * np.cos(np.random.rand() * np.pi) + np.random.normal(0, 0.5))
            acc_z.append(1.8 * np.sin(np.random.rand() * np.pi) + np.random.normal(0, 0.3))
        elif activity == 'resting':
            acc_x.append(np.random.normal(0, 0.1))
            acc_y.append(np.random.normal(0, 0.1))
            acc_z.append(np.random.normal(0, 0.1))
        else:  
            acc_x.append(np.random.normal(0, 0.8))
            acc_y.append(np.random.normal(0, 0.8))
            acc_z.append(np.random.normal(0, 0.8))
    
    df = pd.DataFrame({
        'timestamp': timestamps,
        'acc_x': acc_x,
        'acc_y': acc_y,
        'acc_z': acc_z,
        'activity': activity_labels
    })
    
    return df

# Generate and display sample data
df = generate_sample_data()
print(f"Generated dataset shape: {df.shape}")
print("\nSample data:")
print(df.head())

print("\nActivity distribution:")
print(df['activity'].value_counts())


#  Data Preprocessing

def preprocess_data(df):
    # Detect and handle outliers
    z_scores = stats.zscore(df[['acc_x', 'acc_y', 'acc_z']])
    abs_z_scores = np.abs(z_scores)
    filtered_entries = (abs_z_scores < 3).all(axis=1)
    df_filtered = df[filtered_entries]
    
    print(f"Removed {len(df) - len(df_filtered)} outliers")
    
    # Apply low-pass filter to remove high-frequency noise
    def butter_lowpass_filter(data, cutoff=3.0, fs=10.0, order=2):
        nyq = 0.5 * fs
        normal_cutoff = cutoff / nyq
        b, a = butter(order, normal_cutoff, btype='low', analog=False)
        y = filtfilt(b, a, data)
        return y
    
    # Apply filter to each accelerometer axis
    for axis in ['acc_x', 'acc_y', 'acc_z']:
        df_filtered[f"{axis}_filtered"] = butter_lowpass_filter(df_filtered[axis].values)
    
    return df_filtered

# Apply preprocessing
df_clean = preprocess_data(df)
print("\nCleaned data sample:")
print(df_clean.head())


# Feature Engineering


def create_features(df, window_size=20):
    """Extract features from accelerometer data using sliding windows"""
    features = []
    labels = []
    

    for i in range(0, len(df) - window_size, window_size // 2):  # 50% overlap
        window = df.iloc[i:i+window_size]
        
  
        if len(window['activity'].unique()) > 1:
            continue
     
        feature_row = {}
        
        for axis in ['acc_x_filtered', 'acc_y_filtered', 'acc_z_filtered']:
            # Basic statistics
            feature_row[f'{axis}_mean'] = window[axis].mean()
            feature_row[f'{axis}_std'] = window[axis].std()
            feature_row[f'{axis}_max'] = window[axis].max()
            feature_row[f'{axis}_min'] = window[axis].min()
            feature_row[f'{axis}_range'] = window[axis].max() - window[axis].min()
            feature_row[f'{axis}_median'] = window[axis].median()
            
            # Higher order statistics
            feature_row[f'{axis}_skew'] = window[axis].skew()
            feature_row[f'{axis}_kurtosis'] = window[axis].kurtosis()
            
            # Signal dynamics
            feature_row[f'{axis}_rms'] = np.sqrt(np.mean(np.square(window[axis])))
            
            # Zero crossings
            zero_crossings = np.where(np.diff(np.signbit(window[axis])))[0]
            feature_row[f'{axis}_zero_crossings'] = len(zero_crossings)
        
        # Cross-axis features
        feature_row['correlation_xy'] = window['acc_x_filtered'].corr(window['acc_y_filtered'])
        feature_row['correlation_xz'] = window['acc_x_filtered'].corr(window['acc_z_filtered'])
        feature_row['correlation_yz'] = window['acc_y_filtered'].corr(window['acc_z_filtered'])
        
        # Magnitude features
        magnitude = np.sqrt(window['acc_x_filtered']**2 + 
                           window['acc_y_filtered']**2 + 
                           window['acc_z_filtered']**2)
        
        feature_row['magnitude_mean'] = magnitude.mean()
        feature_row['magnitude_std'] = magnitude.std()
        feature_row['magnitude_max'] = magnitude.max()
        
        features.append(feature_row)
        
        labels.append(window['activity'].mode()[0])
    
    feature_df = pd.DataFrame(features)
    
    return feature_df, labels

# Extract features
X, y = create_features(df_clean)
print(f"\nExtracted {X.shape[1]} features from {X.shape[0]} windows")
print("\nFeature sample:")
print(X.head())

# Data Splitting and Scaling

# Split into train, validation and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print(f"\nTraining set: {X_train.shape[0]} samples")
print(f"Validation set: {X_val.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Save scaler for deployment
joblib.dump(scaler, 'marshee_feature_scaler.pkl')

# Feature Selection

# Train a base model to use for feature selection
base_rf = RandomForestClassifier(n_estimators=100, random_state=42)
base_rf.fit(X_train_scaled, y_train)

# Select features based on importance
feature_selector = SelectFromModel(base_rf, threshold="mean")
feature_selector.fit(X_train_scaled, y_train)

# Apply feature selection
X_train_selected = feature_selector.transform(X_train_scaled)
X_val_selected = feature_selector.transform(X_val_scaled)
X_test_selected = feature_selector.transform(X_test_scaled)

print(f"\nSelected {X_train_selected.shape[1]} out of {X_train_scaled.shape[1]} features")

# Get selected feature names for interpretation
selected_indices = feature_selector.get_support(indices=True)
selected_features = X.columns[selected_indices]
print("\nTop selected features:")
print(selected_features.tolist())

# Model Training and Hyperparameter Tuning

# Define parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Set up K-fold cross-validation
cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Initialize GridSearchCV
grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    cv=cv,
    scoring='f1_weighted',
    verbose=0,
    n_jobs=-1
)

# Perform grid search
print("\nPerforming grid search for hyperparameter tuning...")
grid_search.fit(X_train_selected, y_train)

# Best parameters and score
print(f"\nBest parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_:.4f}")

# Train final model with best parameters
best_rf = grid_search.best_estimator_
best_rf.fit(X_train_selected, y_train)

# Save model for deployment
joblib.dump(best_rf, 'marshee_activity_classifier.pkl')
joblib.dump(feature_selector, 'marshee_feature_selector.pkl')

# Model Evaluation

# Function to evaluate model
def evaluate_model(model, X, y, dataset_name):
    # Make predictions
    y_pred = model.predict(X)
    
    # Calculate metrics
    accuracy = accuracy_score(y, y_pred)
    f1 = f1_score(y, y_pred, average='weighted')
    precision = precision_score(y, y_pred, average='weighted')
    recall = recall_score(y, y_pred, average='weighted')
    
    print(f"\n{dataset_name} Performance:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    
    # Confusion matrix
    cm = confusion_matrix(y, y_pred)
    
    return accuracy, f1, cm, y_pred

# Evaluate on training set
train_accuracy, train_f1, train_cm, train_pred = evaluate_model(
    best_rf, X_train_selected, y_train, "Training Set")

# Evaluate on validation set
val_accuracy, val_f1, val_cm, val_pred = evaluate_model(
    best_rf, X_val_selected, y_val, "Validation Set")

# Evaluate on test set
test_accuracy, test_f1, test_cm, test_pred = evaluate_model(
    best_rf, X_test_selected, y_test, "Test Set")

# Visualization

# Plot confusion matrix
def plot_confusion_matrix(cm, classes, title):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    
    return plt

# Get class labels
class_labels = np.unique(y)

# Plot confusion matrix for test set
cm_plot = plot_confusion_matrix(test_cm, class_labels, 'Confusion Matrix - Test Set')
plt.savefig('confusion_matrix.png')

# Feature importance plot
def plot_feature_importance(model, feature_names):
    # Get feature importances
    importances = model.feature_importances_
    
    # Sort feature importances in descending order
    indices = np.argsort(importances)[::-1]
    
    # Rearrange feature names so they match the sorted feature importances
    names = [feature_names[i] for i in indices]
    
    # Create plot
    plt.figure(figsize=(12, 8))
    plt.title("Feature Importance")
    plt.bar(range(X_train_selected.shape[1]), importances[indices])
    plt.xticks(range(X_train_selected.shape[1]), names, rotation=90)
    plt.tight_layout()
    
    return plt

# Plot feature importance
importance_plot = plot_feature_importance(best_rf, selected_features)
plt.savefig('feature_importance.png')

# Model Conversion for Deployment

# Create a simplified model for on-device deployment
# In a real scenario, we would use TensorFlow Lite or similar

# Select top 10 features for a lighter model
top_n_features = 10
top_feature_indices = np.argsort(best_rf.feature_importances_)[::-1][:top_n_features]

# Train a smaller model with fewer estimators and only top features
deployment_model = RandomForestClassifier(n_estimators=50, 
                                          max_depth=10,
                                          random_state=42)

# Get indices of top features in the original feature set
top_features_in_original = [selected_indices[i] for i in top_feature_indices]
X_train_deployment = X_train_scaled[:, top_features_in_original]
X_test_deployment = X_test_scaled[:, top_features_in_original]

# Train the deployment model
deployment_model.fit(X_train_deployment, y_train)

# Evaluate deployment model
print("\nEvaluating lightweight deployment model:")
deploy_accuracy, deploy_f1, deploy_cm, deploy_pred = evaluate_model(
    deployment_model, X_test_deployment, y_test, "Deployment Model (Test Set)")

# Save deployment model
joblib.dump(deployment_model, 'marshee_deployment_model.pkl')
joblib.dump(top_features_in_original, 'marshee_deployment_feature_indices.pkl')

# 10. Real-time Inference Simulation

def simulate_realtime_processing(raw_data, window_size=20, overlap=10):
    """Simulate real-time processing of accelerometer data"""
    # Apply preprocessing to raw data
    processed_data = preprocess_data(raw_data)
    
    results = []
    
    # Process data in overlapping windows
    for i in range(0, len(processed_data) - window_size, overlap):
        # Extract window
        window = processed_data.iloc[i:i+window_size]
        
        # Extract features (simplified for demonstration)
        window_features = {}
        
        for axis in ['acc_x_filtered', 'acc_y_filtered', 'acc_z_filtered']:
            window_features[f'{axis}_mean'] = window[axis].mean()
            window_features[f'{axis}_std'] = window[axis].std()
            # Add other feature extraction here...
        
        # Convert to DataFrame and align with training features
        features_df = pd.DataFrame([window_features])
        aligned_features = pd.DataFrame(columns=X.columns)
        for col in features_df.columns:
            if col in aligned_features.columns:
                aligned_features[col] = features_df[col]
        
        aligned_features = aligned_features.fillna(0)
        
        # Scale features
        scaled_features = scaler.transform(aligned_features)
        
        # Select only the features needed for the deployment model
        selected_features = scaled_features[:, top_features_in_original]
        
        # Make prediction
        prediction = deployment_model.predict(selected_features)[0]
        probability = np.max(deployment_model.predict_proba(selected_features)[0])
        
        # Store result
        results.append({
            'start_time': window.iloc[0]['timestamp'],
            'end_time': window.iloc[-1]['timestamp'],
            'prediction': prediction,
            'confidence': probability
        })
    
    return pd.DataFrame(results)

# Generate some new raw data to simulate real-time processing
new_raw_data = generate_sample_data(1000)

# Simulate real-time processing
print("\nSimulating real-time processing...")
realtime_results = simulate_realtime_processing(new_raw_data)
print("\nReal-time processing results:")
print(realtime_results.head())

# 11. Summary and Final Metrics
z
print("\n===== Model Performance Summary =====")
print(f"Training Accuracy: {train_accuracy:.4f}, F1: {train_f1:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}, F1: {val_f1:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}, F1: {test_f1:.4f}")
print(f"Deployment Model Test Accuracy: {deploy_accuracy:.4f}, F1: {deploy_f1:.4f}")
print(f"Selected Features: {X_train_selected.shape[1]} out of {X_train_scaled.shape[1]}")
print(f"Deployment Model Features: {top_n_features}")
print("\nModel training and evaluation complete.")